In [1]:
from flowerpower.transform import Transformer


class Test(Transformer):

    @staticmethod
    def _1_func1(input:dict, a:int, **kargs):
        output = {k:v+a for k,v in input.items()}
        return output
    
    @staticmethod 
    def _2_func2(input:dict, b:int, **kwargs):
        output = {k:v+b for k,v in input.items()}
        return output



In [2]:
t = Test()

In [3]:
t.transform({'a':1,'b':2})

{'a': 4, 'b': 5}

In [49]:
from dataclasses import dataclass, make_dataclass, asdict
from tkinter import N
from typing import Any
import toml
import yaml
import json
import os


def dict_to_dataclass(d) -> Any:
    fields = {
        k: (dict_to_dataclass(v) if isinstance(v, dict) else v) for k, v in d.items()
    }
    DataClass = make_dataclass("DataClass", fields=fields)
    return DataClass(**fields)

class Params:
    def __init__(self, cfg_path: str="conf") -> Any:
        self._cfg_path = cfg_path
        self._params = self._load_params()

    def __getattr__(self, name:str|None)->Any:
        return self._params if name is None else self._params.__getattribute__(name)
    
    def __get__(self, name:str|None)->Any:
        return self._params if name is None else self._params.__getattribute__(name)
    
    def get(self, name:str|None, asdict:bool=False)->Any|dict:
        if asdict:
            return asdict(self._params) if name is None else asdict(getattr(self,name))
        return self._params if name is None else self._params.__getattribute__(name)
    
    def __getitem__(self, name:str|None)->Any:
        return self._params if name is None else self._params.__getattribute__(name)
    
    
    def asdict(self, name:str|None=None)->dict:
       
        return asdict(self._params) if name is None else asdict(getattr(self,name))

    def attrs(self,name:str|None=None)->tuple:
        return tuple(self._params.__dict__.keys()) if name is None else tuple(getattr(self,name).__dict__.keys())


    def _load_params(self) -> Any:
        params_path = os.path.join(self._cfg_path, "params")

        if os.path.exists(params_path + ".toml"):
            with open(params_path + ".toml") as f:
                return dict_to_dataclass(toml.load(f))

        elif os.path.exists(params_path + ".yaml"):
            with open(params_path + ".yaml") as f:
                return dict_to_dataclass(yaml.safe_load(f))

        elif os.path.exists(params_path + ".yml"):
            with open(params_path + ".yml") as f:
                return dict_to_dataclass(yaml.safe_load(f))

        elif os.path.exists(params_path + ".json"):
            with open(params_path + ".json") as f:
                return dict_to_dataclass(json.load(f))

        raise Exception(f"Could not find params at {params_path}")

pp=Params("conf")

TypeError: asdict() should be called on dataclass instances

In [48]:
asdict(pp["test"])

{'_1_func1': {'a': 1, 'c': 3}, '_2_func2': {'c': 3, 'b': 2}}

In [23]:
from typing import Any

@dataclass
class A:
    a:int
    b:int

    def __get__(self, __name: str) -> Any:
        return self.__dict__[__name]

In [24]:
a=A(a=1,b=1)

In [26]:
a.get("a")

RecursionError: maximum recursion depth exceeded

In [32]:
params

DataClass(_1_func1=DataClass(a=1), _2_func2=DataClass(b=2))